In [1]:
import pandas as pd

In [5]:


import pandas as pd

# Define the chunk size
chunk_size = 1000000 

chunks = []

for chunk in pd.read_csv("products_data.csv",on_bad_lines='skip',sep='\t', chunksize=chunk_size):
    chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
df = pd.concat(chunks, ignore_index=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18472\3223727445.py:13: DtypeWarning: Columns (0,11,17,31,32,33,35,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("products_data.csv",on_bad_lines='skip',sep='\t', chunksize=chunk_size):
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18472\3223727445.py:13: DtypeWarning: Columns (0,11,17,33,34,35,45,50,52,53,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("products_data.csv",on_bad_lines='skip',sep='\t', chunksize=chunk_size):
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18472\3223727445.py:13: DtypeWarning: Columns (0,11,17,33,35,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("products_data.csv",on_bad_lines='skip',sep='\t', chunksize=chunk_size):
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18472\3223727445.py:13: DtypeWarning: Columns (0,11,17,33,35,53,67,72)

In [6]:
#Loading only the qualified 
df_qualified = df[~(df['image_nutrition_url'].isna())]

In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import nltk
from nltk.corpus import stopwords
import string

In [10]:
df_qualified.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime',
       ...
       'glycemic-index_100g', 'water-hardness_100g', 'choline_100g',
       'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g',
       'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 'acidity_100g'],
      dtype='object', length=206)

In [8]:
# Download NLTK stopwords
nltk.download('stopwords', quiet=True)

# Function to get stopwords for multiple languages
def get_multilingual_stopwords():
    languages = ['english', 'spanish', 'french', 'german', 'italian', 'portuguese', 'dutch']
    all_stopwords = set()
    for lang in languages:
        all_stopwords.update(stopwords.words(lang))
    return list(all_stopwords)

In [9]:
def multilingual_tokenizer(text):
    # Remove punctuation and convert to lowercase
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text.split()

In [11]:
product_names = df_qualified['product_name'].astype(str)

In [12]:
len(product_names)

1828041

In [13]:
# Get multilingual stopwords
multilingual_stopwords = get_multilingual_stopwords()

In [14]:
nltk.download('stopwords', quiet=True)

True

In [15]:
vectorizer = TfidfVectorizer(
    tokenizer=multilingual_tokenizer,
    stop_words=multilingual_stopwords,
    ngram_range=(1, 3)  # Consider both unigrams and bigrams
)

In [16]:
# Fit and transform the product names
tfidf_matrix = vectorizer.fit_transform(product_names)


C:\Users\LENOVO\anaconda3\envs\myenv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\LENOVO\anaconda3\envs\myenv\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [17]:
# Normalize the TF-IDF matrix
normalized_tfidf = normalize(tfidf_matrix)




In [18]:
normalized_tfidf

<1828041x2258744 sparse matrix of type '<class 'numpy.float64'>'
	with 9856681 stored elements in Compressed Sparse Row format>

In [19]:
# Perform K-means clustering (adjust n_clusters as needed)
n_clusters = 5  # You can change this number based on your needs
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(normalized_tfidf)

KMeans(n_clusters=5, n_init=10, random_state=42)

In [20]:
# Add cluster labels to the original dataframe
df_qualified['cluster'] = kmeans.labels_

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_18472\4018986303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_qualified['cluster'] = kmeans.labels_


In [21]:
df_qualified.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,...,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g,cluster
1,0000000000000207025004,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1656948610,2022-07-04T15:30:10Z,1656948613,2022-07-04T15:30:13Z,kiliweb,1.707864e+09,2024-02-13T22:43:38Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1630483911,2021-09-01T08:11:51Z,1682646029,2023-04-28T01:40:29Z,isabel626,1.707844e+09,2024-02-13T17:00:47Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,00000000000026772226,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1654250311,2022-06-03T09:58:31Z,1654270474,2022-06-03T15:34:34Z,quentinbrd,1.707742e+09,2024-02-12T12:49:37Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1624390765,2021-06-22T19:39:25Z,sylvariane,1.707637e+09,2024-02-11T07:29:45Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,000000000004128579,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1649449251,2022-04-08T20:20:51Z,1649449920,2022-04-08T20:32:00Z,roboto-app,1.707859e+09,2024-02-13T21:11:00Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [22]:

# Print the top terms for each cluster
feature_names = np.array(vectorizer.get_feature_names_out())
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]


In [23]:
print("Top terms per cluster:")
for i in range(n_clusters):
    print(f"Cluster {i}:")
    for ind in order_centroids[i, :10]:
        print(f" {feature_names[ind]}", end='')
    print('\n')


Top terms per cluster:
Cluster 0:
 chocolate bio sauce extra queso poulet chicken milk chips protein

Cluster 1:
 yogurt greek greek yogurt intero yogurt intero magro yogurt magro greco yogurt greco bianco

Cluster 2:
 lait chocolat lait chocolat gras foie gras foie gras canard entier foie gras canard canard

Cluster 3:
 nan 🫒 feel good kefir feel good slim feel good selection feel good proteine feel good protein feel good organic feel good mix feel good indulgence

Cluster 4:
 chocolat chocolat noir noir pain chocolat pépites pépites chocolat pain chocolat blanc mousse chocolat blanc



In [27]:
#Cluster Statistics
df_qualified['cluster'].value_counts()

cluster
0    1729506
3      53531
4      20856
2      15499
1       8649
Name: count, dtype: int64